In [1]:
import os
# import yaml

os.environ["CUDA_VISIBLE_DEVICES"] = '3'
# with open("config.yaml", "r") as file:
#     hp = yaml.safe_load(file)

In [1]:
from datasets import Dataset
import pandas as pd

dataset = pd.read_json("dataset/training.json").drop(['explination'], axis = 1)
dataset = Dataset.from_pandas(dataset).train_test_split(test_size=0.2)
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10808
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2702
    })
})

In [ ]:
from transformers import BitsAndBytesConfig, AutoModelForSequenceClassification
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
import torch
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, 
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True, 
    bnb_4bit_compute_dtype = torch.float16 
)
model_name = 'taide/Llama3-TAIDE-LX-8B-Chat-Alpha1'
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels=2,
    device_map='auto',
    cache_dir = '/HDD/model_cache'
)
lora_config = LoraConfig(
    r = 4, 
    lora_alpha = 8,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.1, 
    bias = 'none',
    task_type = 'SEQ_CLS'
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def data_preprocesing(row):
    return tokenizer(row['text'], truncation=True, max_length=512)

tokenized_data = dataset.map(data_preprocesing, batched=True, 
remove_columns=['text'])
tokenized_data.set_format("torch")
tokenized_data

In [41]:
import evaluate
import numpy as np
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return f1_metric.compute(predictions=predictions, references=labels, average="micro")

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

training_args = TrainingArguments(
    output_dir = 'checkpoint',
    learning_rate = 1e-4,
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    max_steps = 1000,
    logging_steps = 1,
    weight_decay = 0.01,
    eval_strategy = 'steps',
    save_strategy = 'steps',
    eval_steps = 200,
    save_steps = 200,
    warmup_steps=100,
    load_best_model_at_end = True,
    report_to="none",
    optim="adamw_torch"
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding=True),
    eval_dataset=tokenized_data['test'],
    compute_metrics=compute_metrics,
)

In [ ]:
model.config.use_cache=False
model.config.pad_token_id = tokenizer.pad_token_id
model.train()
trainer.train()

In [44]:
model.save_pretrained('fine-tuned model')

### Inference

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import AutoPeftModelForSequenceClassification, PeftModel

base_model_name = 'taide/Llama3-TAIDE-LX-8B-Chat-Alpha1'
adapter_model_path = './fine-tuned model'
tokenizer = AutoTokenizer.from_pretrained(
    base_model_name, 
    add_prefix_space=True, 
    cache_dir = '/HDD/model_cache'
)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token
base_model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    cache_dir = '/HDD/model_cache',
    num_labels=2
)

peft_model = PeftModel.from_pretrained(base_model, adapter_model_path)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at taide/Llama3-TAIDE-LX-8B-Chat-Alpha1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from transformers import pipeline
peft_model.eval()
classifier = pipeline(
    model=peft_model,
    tokenizer=tokenizer,
    task='sentiment-analysis',
    device=0,
)
classifier('我有五百萬')

The model 'PeftModelForSequenceClassification' is not supported for sentiment-analysis. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSequenceClassification',

[{'label': 'LABEL_0', 'score': 0.5042698383331299}]

In [ ]:
import torch
import evaluate
from transformers import AutoTokenizer
from peft import AutoModelForSequenceClassification
inf_model_name = "fine-tuned model"
base_model = 'meta-llama/Meta-Llama-3-8B'
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    cache_dir='/HDD/model_cache'
)
inf_model = AutoPeftModelForSequenceClassification.from_pretrained(
    inf_model_name,
    cache_dir='/HDD/model_cache',
    num_labels=4
)


In [ ]:
from transformers import AutoTokenizer
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token
def data_preprocesing(row):
    return tokenizer(row['text'], truncation=True, max_length=512)

tokenized_test_data = test_data.map(data_preprocesing, batched=True, 
remove_columns=['text'])
tokenized_test_data.set_format("torch")
tokenized_test_data

In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import DataCollatorWithPadding
import os
import evaluate

os.environ["TOKENIZERS_PARALLELISM"] = "false"
test_loader = DataLoader(
    tokenized_test_data, 
    batch_size=128, 
    collate_fn=DataCollatorWithPadding(tokenizer=tokenizer, padding=True),
    num_workers=16
)

inf_model.eval()
inf_model.config.pad_token_id = tokenizer.pad_token_id
f1_metric = evaluate.load("f1")
all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating", pin_memory=True):
        batch = {k: v.to(inf_model.device) for k, v in batch.items()}
        
        outputs = inf_model(**batch)
        logits = outputs.logits
        
        predictions = torch.argmax(logits, dim=-1)
        
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

f1 = f1_metric.compute(predictions=all_predictions, references=all_labels, average='weighted')  # 或者 'macro'

print(f"F1 Score on Test Data: {f1}")
